In [ ]:
import os, glob, subprocess, sys
sys.path.append('/home/phorton/Barefoot_Rover/src/ML')
import numpy as np
import tools_pg as tools
import plots_pg as plots
from matplotlib import pyplot as plt
from skimage.feature import peak_local_max
from sklearn.neighbors import KDTree
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from ipywidgets import interact, Layout, IntSlider
%matplotlib inline

Below we load up all the files that do not contain a rock positions file that are from the rock directories

In [ ]:
datadir = '/data/MLIA_active_data/data_barefoot/'
subfolders = ['train/rock_detection','data_Interns']
files = np.hstack([glob.glob(datadir + a + '/*rock-*/') for a in subfolders])
plotmaindir = datadir + 'train/' 
complete_mask = np.array([len(glob.glob(file + "/rock_positions_*")) == 1 for file in files])
files = np.sort(files[~complete_mask])
processing = 0

We then fix any errors with the date format. Also some files are missing photos so we remove them from our process list

In [ ]:
bad_files = []
# Fixes all image file names
for file in files:
    fname = file.split('/')[-2].split('_')
    ftype = '_'.join([fname[a] for a in [1,3, 5, 9, 13, 15]])
    jpgs = glob.glob(file +'/*[0-9]/*.jpg')
    if len(jpgs) > 0:
        jpg_time = np.hstack([a.split('/')[-1].split('.')[0] for a in jpgs])
        for j in range(len(jpg_time)):
            name = jpg_time[j].split('_')[-1]
            if len(name) < 6:
                new_name = ''.join([name, ''.join(['0']* (6 - len(name)))])
                name_split = jpgs[j].split('_')
                name_split[-1] = new_name + '.jpg'
                jpg_new = '_'.join(name_split)
                os.rename(jpgs[j], jpg_new)
    else:
        bad_files.append(file)
        print("WARNING:", file, "is missing photos... removing!")
for file in bad_files:
    files = np.delete(files,np.where(files == file))

We make a funciton to load the image if we want to use the interactive route to identify files

In [ ]:
def load_img(i):
    plt.figure(figsize=(10,5))
    global current_file
    current_file = jpgs[i]
    plt.imshow(mpimg.imread(current_file))
    plt.show()

Once we have everything loaded we start from here. This will load all the image names for a given processing ID (starting at 0 and incrementing after every process). Once the images are all loaded we need to identify the interesting bits. 

In [ ]:
file = files[processing]
print(f'Processing {processing} of {len(files)}: {file}')
fname = file.split('/')[-2].split('_')
ftype = '_'.join([fname[a] for a in [1,3, 5, 9, 13, 15]])
jpgs = glob.glob(file +'/*[0-9]/*.jpg')
jpgs = np.sort(jpgs)
jpg_time = np.hstack([a.split('/')[-1].split('.')[0] for a in jpgs])
jpg_time = np.sort(jpg_time)

To identify the interesting bits we can either use this function to scroll through the images or use your computers image viewer to view the images on the server. 

Using the widget below you can scroll through the images. When you are on an image of interest use the next cell to print out the current file name and place it in the rocks
array below. 

In [ ]:
interact(load_img,i=widgets.IntSlider(min=0,max=len(jpgs)-1,step=1,layout=Layout(width="1000px")))

In [ ]:
current_file.split('/')[-1].split('.')[0]

The rocks array can be used to identify start, stop, and depth of points of interest. It is formatted like so:

```
rocks = [
    (start_name, stop_name, depth),
    (start_name, stop_name, depth),
    ...
]
```

The names of the start and stop can be obtained from the above widget or from your computers file browser

In [ ]:
rocks = [
    ("start","stop",0.0),
    ("start","stop",0.0)
]

Finally we create a text file with the new information. Check if the indicies of the start and end point for each rock make sense. If they do run the proceeding cell to save the file to the directory.

In [ ]:
text = np.column_stack([jpg_time] + 
                       [np.repeat(0, len(jpg_time))]*(len(rocks) + 1) + 
                       [np.repeat(0.0, len(jpg_time))])
for i, (start, end, depth) in enumerate(rocks):
    i1 = int(np.where(text[:,0] == start)[0])
    i2 = int(np.where(text[:,0] == end)[0])
    print(i1,i2)
    text[i1:i2,i+1] = '1'
    text[i1:i2,-1] = str(depth)

In [ ]:
np.savetxt(file + 'rock_positions_' + ftype + '.txt', text, fmt='%s')

Increment the processing ID and go back to the cell that starts with `file=files[processing]` to start labeling the next rock

In [ ]:
processing += 1